# Get data from Amazon product co-purchasing network metadata

In [1]:
from src.parser import *
import simplejson
import subprocess

from src.utils import select_from_fulldump
from src.insert_data import insert_from_dict_gen
import matplotlib.pyplot as plt

from src.query import q_a, q_c, q_d, q_e, q_g

In [2]:
file_path = "data/amazon-meta.txt.gz"
result = subprocess.Popen('wc -l data/amazon-meta.txt.gz', shell=True, stdout=subprocess.PIPE)
n_lines = int([l for l in result.stdout][0].decode().split()[0])

dumps = [e for e in parse(file_path, total=n_lines) if e]
for dump in dumps:
    if 'group' in dump:
        dump['product_group'] = dump.pop('group')
n_dumps = len(dumps)

print('lines:', n_lines)
print('number of dumps:', n_dumps)
# print(simplejson.dumps(dumps[1], indent=2))

15010574it [00:38, 392091.06it/s]                          


lines: 770519
number of dumps: 548552


In [3]:
dumps[1]

{'id': '1',
 'ASIN': '0827229534',
 'title': 'Patterns of Preaching: A Sermon Sampler',
 'salesrank': '396585',
 'similar_amount': '5',
 'similar_items': ['0804215715',
  '156101074X',
  '0687023955',
  '0687074231',
  '082721619X'],
 'categories_amount': '2',
 'reviews_total': 2,
 'reviews_downloaded': 2,
 'reviews_avg_rating': 5.0,
 'reviews': [{'time': '2000-7-28',
   'customer_id': 'A2JW67OY8U6HHK',
   'rating': 5,
   'votes': 10,
   'helpful': 9},
  {'time': '2003-12-14',
   'customer_id': 'A2VE83MZF98ITY',
   'rating': 5,
   'votes': 6,
   'helpful': 5}],
 'categories': [{'id': 12368,
   'hierarchy': '|Books[283155]|Subjects[1000]|Religion & Spirituality[22]|Christianity[12290]|Clergy[12360]|Preaching[12368]'},
  {'id': 12370,
   'hierarchy': '|Books[283155]|Subjects[1000]|Religion & Spirituality[22]|Christianity[12290]|Clergy[12360]|Sermons[12370]'}],
 'product_group': 'Book'}

# insert dumps into psql

In [ ]:
!python3 src/create_schema.py

# number os dumps
n = n_dumps

prods = select_from_fulldump(dumps, 'products', n)
cats = select_from_fulldump(dumps, 'categories', n)
revs = select_from_fulldump(dumps, 'reviews', n)

insert_from_dict_gen('products', prods)
insert_from_dict_gen('categories', cats)
insert_from_dict_gen('reviews', revs)

412335it [00:40, 10174.80it/s]

# Dash BOard

(a) Dado produto, listar os 5 comentários mais úteis e com maior avaliação e os 5 comentários mais úteis e com menor avaliação

(b) Dado um produto, listar os produtos similares com maiores vendas do que ele

(c) Dado um produto, mostrar a evolução diária das médias de avaliação ao longo do intervalo de tempo coberto no arquivo de entrada

(d) Listar os 10 produtos lideres de venda em cada grupo de produtos

(e) Listar os 10 produtos com a maior média de avaliações úteis positivas por produto

(f) Listar a 5 categorias de produto com a maior média de avaliações úteis positivas por produto

(g) Listar os 10 clientes que mais fizeram comentários por grupo de produto

# A

In [ ]:
product_id = 8
a = q_a(product_id)
a.head(10)

In [ ]:
a[::-1].plot.barh(title='helpful comments', x='customer_id', y='helpful');

# C

In [ ]:
c = q_c(product_id)
c.head(10)

In [ ]:
c.plot(title='product avg rating', x='time', y='avg_rating');

# D

In [ ]:
d = q_d()
d.head(10)

In [ ]:
d[::-1][:10].plot.barh(title='Top Ranked Groups', x='product_group', y='salesrank')
plt.show()

# E

In [ ]:
e = q_e()
e.head(10)

In [ ]:
e.plot.barh(title='Top reviews avgrating', x='title', y='reviews_avg_rating')

# G

In [ ]:
g = q_g()
g.head(10)